Notebook pour le projet 3 d'OpenClassrooms track Data Analyst

Les données utilisées proviennent de : http://www.fao.org/faostat/fr/#data/FBS
Bilans Alimentaires: Région Monde Liste, Tous les éléments, Produits (Population), 
                    Groupe de produit (Produits végétaux et animaux Liste), Année 2012 et 2013
Sécurité Alimentaire: Région Monde Liste, Element: Valeur, Produits: Nombre de personnes sous-alimentées, Année 2012-2017

In [8]:
import numpy as np
import pandas as pd

# Lecture des fichiers csv.
# Bilan alimentaire
# On peut facilement ajouter des années
df_al = pd.read_csv('bilan_alimentaire_2013.csv', sep=',')
#Sécurité alimentaire
df_se = pd.read_csv('securite_alimentaire_2012_2017.csv', sep=',')

In [9]:
# Création de la table 'df_unite' contenant les unités par élément.
df_unite = df_al.loc[:,["Élément","Unité"]]
df_unite = df_unite.drop_duplicates(subset=["Élément"])
df_unite = df_unite.reset_index(drop=True)
df_unite.head()

,Élément,Unité
0,Population totale,1000 personnes
1,Production,Milliers de tonnes
2,Importations - Quantité,Milliers de tonnes
3,Variation de stock,Milliers de tonnes
4,Disponibilité intérieure,Milliers de tonnes


In [10]:
# Pour notre pivot table
# Index sera les colonnes de gauche permettant de définir nos lignes
# Values pour définir les colonnes auxquelles on tient
# sum or np.sum pour éviter un average
# column optionnal mais permet de segmenter par cette colonne
# Population dans Population totale
# On ne met pas la colonne unité car sur 4 lignes une seule utile
df_al = df_al.pivot_table(index=["Code Pays","Pays","Code Produit","Produit","Année"], values='Valeur', aggfunc=sum, columns =["Élément"])
df_al = df_al.reset_index()
df_al.head()

Élément,Code Pays,Pays,Code Produit,Produit,Année,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Population totale,Production,Semences,Traitement,Variation de stock
0,1,Arménie,2501,Population,2013,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2977.0,NaN,NaN,NaN,NaN
1,1,Arménie,2511,Blé,2013,93.0,0.0,1024.0,130.60,3.6,...,554.0,1.0,361.0,389.0,32.0,NaN,312.0,30.0,10.0,-118.0
2,1,Arménie,2513,Orge,2013,137.0,26.0,0.0,0.00,0.0,...,198.0,0.0,9.0,0.0,15.0,NaN,189.0,14.0,7.0,0.0
3,1,Arménie,2514,Maïs,2013,96.0,NaN,0.0,0.03,NaN,...,102.0,NaN,82.0,0.0,7.0,NaN,21.0,0.0,NaN,NaN
4,1,Arménie,2515,Seigle,2013,1.0,NaN,1.0,0.12,0.0,...,1.0,NaN,0.0,0.0,0.0,NaN,1.0,0.0,NaN,0.0


In [5]:
# On créée une nouvelle table. On sélectionne les colonnes que l'on souhaite.
df_population = df_al.loc[:,["Code Pays","Pays","Année","Produit", "Population totale"]]
# On filtre sur les produits pour garder les populations
df_population = df_population[df_population['Produit'].isin(['Population'])]
# On multiplie par 1000 la colonne population
df_population['Population totale'] *= 1000 
# On enlève le zéro inutile
df_population = df_population.astype({'Population totale':int})
df_population = df_population.reset_index(drop=True)
df_population.head()


Élément,Code Pays,Pays,Année,Produit,Population totale
0,1,Arménie,2013,Population,2977000
1,2,Afghanistan,2013,Population,30552000
2,3,Albanie,2013,Population,3173000
3,4,Algérie,2013,Population,39208000
4,7,Angola,2013,Population,21472000


### Question 1
Calculez le nombre total d’humains sur la planète pour l'année 2013.

In [6]:
population_totale = df_population["Population totale"].sum()
print("La population totale pour l'année 2013 est de {} humains.".format(population_totale))
print("L'ONU déclare une population de plus de 7,2 milliards pour l'année 2013. \nSource: https://population.un.org/wpp/DataQuery/")

La population totale pour l'année 2013 est de 6997326000 humains.
L'ONU déclare une population de plus de 7,2 milliards pour l'année 2013. 
Source: https://population.un.org/wpp/DataQuery/


### Question 2
Parmi les documents sur les Bilans alimentaires que vous avez téléchargés, 
il y a des informations redondantes. 
En effet, certaines peuvent se calculer à partir d'autres :

    Production (Production)
    Importations (Import Quantity)
    Exportations (Export Quantity)
    Variation de stock (Stock Variation)
    Disponibilité intérieure (Domestic supply)
    Semences (Seed)
    Pertes (Waste)
    Nourriture (Food) , aussi appelée Disponibilité alimentaire (Food Supply)
    Aliments pour animaux (Feed)
    Traitement (Processing)
    Autres utilisations (Other uses)

Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique. 
C'est une équation à 3 termes de type a1+a2+[...]=b1+b2+[...]=c1+c2+[...]) 
faisant intervenir chacune des 11 quantités données ci dessus. 
Illustrez cette équation avec l'exemple du blé en France.


### Equation

Production + Importation - Exportation + Variation de stock = 
Disponibilité intérieure =
Semences + Pertes + Nourriture + Aliments pour anaimaux + Traitement + Autres Utilisations


In [7]:
# Pour illustrer notre formule nous utilisons l'exemple du blé en France.
# On filtre sur les lignes pour obtenir seulement la ligne du blé français.
df_al_temp = df_al[(df_al["Produit"]=="Blé") & (df_al["Pays"]=="France")]
# On filtre les colonnes qui nous intéressent
df_al_temp.loc[:,["Pays", "Produit","Production", "Importations - Quantité", "Exportations - Quantité", 
                "Variation de stock", "Disponibilité intérieure", "Semences", "Pertes", 
                "Nourriture", "Aliments pour animaux", "Traitement", "Autres Utilisations"]]

Élément,Pays,Produit,Production,Importations - Quantité,Exportations - Quantité,Variation de stock,Disponibilité intérieure,Semences,Pertes,Nourriture,Aliments pour animaux,Traitement,Autres Utilisations
4191,France,Blé,38614.0,2055.0,21502.0,1131.0,20298.0,748.0,358.0,6971.0,7822.0,1575.0,2824.0


In [8]:
# On calcule chacun des trois termes.
a = (df_al_temp.loc[4191,"Production"] + 
    df_al_temp.loc[4191,"Importations - Quantité"] -
    df_al_temp.loc[4191,"Exportations - Quantité"] +
    df_al_temp.loc[4191,"Variation de stock"])
b = df_al_temp.loc[4191,"Disponibilité intérieure"]
c = (df_al_temp.loc[4191,"Semences"] + 
    df_al_temp.loc[4191,"Pertes"] +
    df_al_temp.loc[4191,"Nourriture"] +
    df_al_temp.loc[4191,"Aliments pour animaux"] +
    df_al_temp.loc[4191,"Traitement"] +
    df_al_temp.loc[4191,"Autres Utilisations"])
# On vérifie l'égalité.
a == b == c

True

### Question 3 
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

Vous ferez cela à partir de ces informations :

    Population de chaque pays ;
    Disponibilité alimentaire ("Food Supply" en anglais) donnée pour chaque produit 
    et pour chaque pays en kcal/personne/jour.
    Disponibilité alimentaire en protéines ("Protein supply quantity" en anglais) 
    donnée pour chaque produit et pour chaque pays en g/personne/jour.
    
    
    

In [9]:
# Nettoyons notre table
# On supprime les lignes en lien avec la population car nous avons déjà la table population
df_al = df_al[df_al["Produit"]!="Population"]
# On supprime l'ancienne colonne population totale.
df_al = df_al.drop(columns = "Population totale")
df_al = df_al.reset_index(drop=True)
# On ajoute une colonne population totale.
df_al = pd.merge(df_al, df_population[["Code Pays", "Année", "Population totale"]], how="inner", on=["Code Pays", "Année"])
df_al.head()
# On remplace nos valeurs infinies par NaN.
df_al.replace([np.inf, -np.inf],value = np.nan, inplace = True)
# On remplace nos valeurs NaN par 0.
df_al.fillna(0, inplace = True)

In [10]:
# On calcule deux nouvelles colonnes
# Disponibilité totale alimentaire (Kcal/jour)
df_al["Disponibilité totale alimentaire (Kcal/jour)"]= df_al["Disponibilité alimentaire (Kcal/personne/jour)"] * df_al["Population totale"]
# Disponibilité totale de protéines en quantité (kg/jour)
df_al["Disponibilité totale de protéines en quantité (Kg/jour)"] = (df_al["Disponibilité de protéines en quantité (g/personne/jour)"] * df_al["Population totale"])/1000
df_al.head()

Élément,Code Pays,Pays,Code Produit,Produit,Année,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,Population totale,Disponibilité totale alimentaire (Kcal/jour),Disponibilité totale de protéines en quantité (Kg/jour)
0,1,Arménie,2511,Blé,2013,93.0,0.0,1024.0,130.60,3.60,...,361.0,389.0,32.0,312.0,30.0,10.0,-118.0,2977000,3.048448e+09,90858.04
1,1,Arménie,2513,Orge,2013,137.0,26.0,0.0,0.00,0.00,...,9.0,0.0,15.0,189.0,14.0,7.0,0.0,2977000,0.000000e+00,0.00
2,1,Arménie,2514,Maïs,2013,96.0,0.0,0.0,0.03,0.00,...,82.0,0.0,7.0,21.0,0.0,0.0,0.0,2977000,0.000000e+00,29.77
3,1,Arménie,2515,Seigle,2013,1.0,0.0,1.0,0.12,0.00,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2977000,2.977000e+06,59.54
4,1,Arménie,2516,Avoine,2013,4.0,0.0,2.0,0.37,0.03,...,1.0,1.0,0.0,5.0,0.0,0.0,0.0,2977000,5.954000e+06,267.93


In [11]:
# On groupe par pays et année.
# On fait la somme des colonnes "Disponibilité alimentaire (Kcal/personne/jour)" et "Disponibilité de protéines en quantité (g/personne/jour)"
df_al_pays = df_al.groupby(["Code Pays", "Pays", "Année"])[["Disponibilité alimentaire (Kcal/personne/jour)", "Disponibilité de protéines en quantité (g/personne/jour)","Disponibilité totale alimentaire (Kcal/jour)","Disponibilité totale de protéines en quantité (Kg/jour)"]].sum()
df_al_pays = df_al_pays.reset_index()
df_al_pays.head()

Élément,Code Pays,Pays,Année,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité totale alimentaire (Kcal/jour),Disponibilité totale de protéines en quantité (Kg/jour)
0,1,Arménie,2013,2924.0,90.06,8.704748e+09,268108.62
1,2,Afghanistan,2013,2087.0,58.26,6.376202e+10,1779959.52
2,3,Albanie,2013,3188.0,111.37,1.011552e+10,353377.01
3,4,Algérie,2013,3293.0,91.92,1.291119e+11,3603999.36
4,7,Angola,2013,2474.0,57.27,5.312173e+10,1229701.44


### Question 4 
A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit),
calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg.
Indication : La disponibilité alimentaire en kcal/personne/jour est calculée par la FAO en multipliant la quantité Nouriture (Nourriture) par le ratio énergie/poids (en kcal/kg), puis en le divisant par la population du pays puis par 365. Ici, on vous demande juste de retrouver le ratio énergie/poids que la FAO a utilisé dans son calcul.
En suivant la même méthodologie, calculez également le pourcentage de protéïnes de chaque produit (pour chaque pays). Ce pourcentage est obtenu en calculant le ratio "poids de protéines/poids total" (attention aux unités utilisées). 

In [12]:
# Ratio "énergie/poids" en kcal/kg pour chaque produit
# On calcule le ratio par produit
# On multiple par 1000*1000 car les données sur la nourriture est en millier de tonnes.
df_al["Ratio énergie/poids en kcal/kg"]= (df_al["Disponibilité totale alimentaire (Kcal/jour)"]*365)/(df_al["Nourriture"]*1000*1000)

# La différence d'un pays à l'autre peut être du aux arrondis.
# Pour les oeufs on s'attendait à une valeur de 1470 kcal par kg.
# Pourcentage de protéines de chaque produit
# On multiple par 1000*1000 car les données sur la nourriture est en millier de tonnes.
df_al["Pourcentage de protéines"]= (df_al["Disponibilité totale de protéines en quantité (Kg/jour)"]*365)/(df_al["Nourriture"]*1000*1000)

df_al[df_al["Produit"]=="Oeufs"].head()
# Pour avoine, on s'attend à une valeur autour de 0.107%.

Élément,Code Pays,Pays,Code Produit,Produit,Année,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Pertes,Production,Semences,Traitement,Variation de stock,Population totale,Disponibilité totale alimentaire (Kcal/jour),Disponibilité totale de protéines en quantité (Kg/jour),Ratio énergie/poids en kcal/kg,Pourcentage de protéines
74,1,Arménie,2744,Oeufs,2013,0.0,0.0,45.0,11.69,3.15,...,0.0,34.0,0.0,0.0,0.0,2977000,1.339650e+08,10211.11,1397.063571,0.106487
144,2,Afghanistan,2744,Oeufs,2013,0.0,0.0,5.0,1.30,0.36,...,5.0,18.0,2.0,0.0,0.0,30552000,1.527600e+08,11609.76,1393.935000,0.105939
226,3,Albanie,2744,Oeufs,2013,0.0,0.0,46.0,12.45,3.28,...,7.0,48.0,0.0,0.0,0.0,3173000,1.459580e+08,11581.45,1331.866750,0.105681
319,4,Algérie,2744,Oeufs,2013,0.0,0.0,27.0,8.06,1.85,...,10.0,347.0,21.0,0.0,0.0,39208000,1.058616e+09,90178.40,1222.768481,0.104162
400,7,Angola,2744,Oeufs,2013,0.0,0.0,4.0,1.11,0.29,...,2.0,5.0,3.0,0.0,0.0,21472000,8.588800e+07,7085.76,1306.213333,0.107763


### Question 5
Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.
Étonnamment, il arrive que ce ratio soit différent en fonction du pays. Il faudra donc réaliser pour chaque aliment une moyenne sur les différents pays. Vous créerez donc une nouvelle table grâce à une agrégation. Attention à bien retirer les valeurs égales à 0 afin de ne pas fausser le calcul de la moyenne.
Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [14]:
df_ratio_classement_1 = df_al.copy()
# On retire les lignes dont leurs valeurs pour Disponibilité alimentaire (Kcal/personne/jour) et/ou Nourriture valent zéro.
df_ratio_classement_1 = df_ratio_classement_1[df_ratio_classement_1["Disponibilité alimentaire (Kcal/personne/jour)"] != 0]
df_ratio_classement_1 = df_ratio_classement_1[df_ratio_classement_1["Nourriture"] != 0]
# On groupe par produit.
df_ratio_classement_1 = df_ratio_classement_1.groupby(["Code Produit","Produit","Année"])[["Ratio énergie/poids en kcal/kg"]].mean()
df_ratio_classement_1 = df_ratio_classement_1.sort_values(by = ["Ratio énergie/poids en kcal/kg"], ascending = False)
df_ratio_classement_1.rename(columns={"Ratio énergie/poids en kcal/kg": "Moyenne Ratio énergie/poids en kcal/kg"}, inplace=True)
df_ratio_classement_1.reset_index(inplace=True)
df_ratio_classement_1.head(20)

Élément,Code Produit,Produit,Année,Moyenne Ratio énergie/poids en kcal/kg
0,2782,Huiles de Foie de Poisso,2013,9982.932500
1,2579,Huile de Sésame,2013,9433.744729
2,2581,Huile de Son de Riz,2013,9361.549883
3,2781,Huiles de Poissons,2013,9003.715504
4,2572,Huile d'Arachide,2013,8935.765811
5,2580,Huile d'Olive,2013,8931.223447
6,2582,Huile de Germe de Maïs,2013,8871.762566
7,2586,Huil Plantes Oleif Autr,2013,8848.222893
8,2576,Huile de Palmistes,2013,8789.540277
9,2575,Huile Graines de Coton,2013,8750.320136


Parmis les aliments les plus caloriques, on peut citer: huile de sésame, huile de poisson, huile d'olive, huile de tournesol et le sésame.

In [15]:
df_ratio_classement_2 = df_al.copy()
# On retire les lignes dont leurs valeurs pour Disponibilité de protéines en quantité (g/personne/jour) et/ou Nourriture valent zéro.
df_ratio_classement_2 = df_ratio_classement_2[df_ratio_classement_2["Disponibilité de protéines en quantité (g/personne/jour)"] != 0]
df_ratio_classement_2 = df_ratio_classement_2[df_ratio_classement_2["Nourriture"] != 0]
# On groupe par produit.
df_ratio_classement_2 = df_ratio_classement_2.groupby(["Code Produit","Produit","Année"])[["Pourcentage de protéines"]].mean()
df_ratio_classement_2 = df_ratio_classement_2.sort_values(by = ["Pourcentage de protéines"], ascending = False)
df_ratio_classement_2.rename(columns={"Pourcentage de protéines":"Moyenne Pourcentage de protéines"},inplace=True)
df_ratio_classement_2.reset_index(inplace=True)
df_ratio_classement_2.head(20)

Élément,Code Produit,Produit,Année,Moyenne Pourcentage de protéines
0,2558,Graines Colza/Moutarde,2013,0.285365
1,2555,Soja,2013,0.285331
2,2556,Arachides Decortiquees,2013,0.251466
3,2547,Pois,2013,0.223244
4,2549,Légumineuses Autres,2013,0.219716
5,2546,Haricots,2013,0.215452
6,2736,Abats Comestible,2013,0.175610
7,2561,Sésame,2013,0.175040
8,2735,"Viande, Autre",2013,0.174024
9,2570,"Plantes Oleiferes, Autre",2013,0.170544


Parmis les aliments les plus riches en protéines, on peut citer: le soja, les pois, les haricots, la viande de bovins, et les piments.
    

### Question 6
Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [16]:
# N'ayant pas la distinction entre les produits végétaux et les produits animaux créont une table contenant cette information.

df_origine = df_al.copy()
# On sélectionne nos colonnes.
df_origine = df_origine.loc[:,["Code Produit", "Produit"]]
df_origine = df_origine.drop_duplicates(subset="Code Produit")
# On ajoute une colonne origine
liste_produits_animaux=["Viande de Bovins","Viande d'Ovins/Caprins","Viande de Suides","Viande de Volailles","Viande, Autre",
                        "Abats Comestible","Beurre, Ghee","Crème","Graisses Animales Crue","Huiles de Poissons","Huiles de Foie de Poisso",
                        "Oeufs","Lait - Excl Beurre","Poissons Eau Douce","Perciform","Poissons Pelagiques","Poissons Marins, Autres","Crustacés",
                        "Cephalopodes","Mollusques, Autres","Viande de Anim Aquatiq","Animaux Aquatiques Autre","Plantes Aquatiques"]
# 23 produits animaux selon la FAO.
# 73 produits végétaux selon la FAO.
# 96 au total selon la FAO.
i = 0
df_origine["Origine"]="Attente_valeur" # On créé la colonne Origine.
for produit in df_origine["Produit"]:
    if produit in liste_produits_animaux:
        df_origine.iloc[i,2]="animale" # On donne une valeur à la colone Origine.
        i += 1
    else:
        df_origine.iloc[i,2]="végétale"
        i += 1
df_origine.reset_index(inplace=True, drop=True)    
df_origine.head()

Élément,Code Produit,Produit,Origine
0,2511,Blé,végétale
1,2513,Orge,végétale
2,2514,Maïs,végétale
3,2515,Seigle,végétale
4,2516,Avoine,végétale


In [7]:
# On merge la table df_al avec df_origine afin d'associer l'origine végétale ou animale des produit.
df_al = pd.merge(df_al, df_origine[["Code Produit", "Origine"]], on = ["Code Produit"]  , how = "inner")
# On merge la table df_al avec df_ratio_classement_1 et df_ratio_classement_2
df_al = pd.merge(df_al, df_ratio_classement_1[["Code Produit", "Moyenne Ratio énergie/poids en kcal/kg"]], on = ["Code Produit"]  , how = "inner")
df_al = pd.merge(df_al, df_ratio_classement_2[["Code Produit", "Moyenne Pourcentage de protéines"]], on = ["Code Produit"]  , how = "inner")

# On remplace les valeurs manquantes NaN par la moyenne.
df_al["Ratio énergie/poids en kcal/kg"].fillna(df_al["Moyenne Ratio énergie/poids en kcal/kg"], inplace=True)
df_al["Pourcentage de protéines"].fillna(df_al["Moyenne Pourcentage de protéines"], inplace=True)

# On remplace les 0 par la valeur moyenne.
df_al["Ratio énergie/poids en kcal/kg"].where(df_al["Ratio énergie/poids en kcal/kg"]!=0, other = df_al["Moyenne Ratio énergie/poids en kcal/kg"], inplace = True)
df_al["Pourcentage de protéines"].where(df_al["Pourcentage de protéines"]!=0, other = df_al["Moyenne Pourcentage de protéines"], inplace = True)

# On remplace nos valeurs infinies par NaN.
df_al.replace([np.inf, -np.inf],value = np.nan, inplace = True)
# On remplace nos valeurs NaN par 0.
df_al.fillna(0, inplace = True)

NameError: name 'df_origine' is not defined

In [18]:
df_al_vegetale = df_al[df_al["Origine"]=="végétale"]
df_al_vegetale.reset_index(inplace=True, drop=True) # On reset l'index pour prendre en compte les lignes éliminées.
df_al_vegetale.head()

Élément,Code Pays,Pays,Code Produit,Produit,Année,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Traitement,Variation de stock,Population totale,Disponibilité totale alimentaire (Kcal/jour),Disponibilité totale de protéines en quantité (Kg/jour),Ratio énergie/poids en kcal/kg,Pourcentage de protéines,Origine,Moyenne Ratio énergie/poids en kcal/kg,Moyenne Pourcentage de protéines
0,1,Arménie,2511,Blé,2013,93.0,0.0,1024.0,130.60,3.60,...,10.0,-118.0,2977000,3.048448e+09,90858.04,2860.368946,0.085252,végétale,2848.688281,0.081866
1,2,Afghanistan,2511,Blé,2013,0.0,0.0,1369.0,160.23,4.69,...,0.0,-350.0,30552000,4.182569e+10,1127674.32,3118.769381,0.084086,végétale,2848.688281,0.081866
2,3,Albanie,2511,Blé,2013,18.0,130.0,1056.0,138.64,5.85,...,0.0,0.0,3173000,3.350688e+09,106739.72,2779.548000,0.088545,végétale,2848.688281,0.081866
3,4,Algérie,2511,Blé,2013,545.0,820.0,1424.0,185.42,5.34,...,0.0,-180.0,39208000,5.583219e+10,1684767.76,2803.129309,0.084586,végétale,2848.688281,0.081866
4,7,Angola,2511,Blé,2013,0.0,0.0,286.0,40.72,1.04,...,0.0,-79.0,21472000,6.140992e+09,186162.24,2564.601922,0.077745,végétale,2848.688281,0.081866


In [19]:
disponibilite_interieur_mondiale_kcal = df_al_vegetale["Disponibilité intérieure"]*1000*1000*df_al_vegetale["Ratio énergie/poids en kcal/kg"]
disponibilite_interieur_mondiale_kcal = np.sum(disponibilite_interieur_mondiale_kcal)
print("La disponibilité intérieur mondiale pour la nourriture végétale est de {} kcal par an.".format(disponibilite_interieur_mondiale_kcal))

La disponibilité intérieur mondiale pour la nourriture végétale est de 1.2867332789576264e+16 kcal par an.


In [20]:
disponibilite_interieur_mondiale_kproteine = df_al_vegetale["Disponibilité intérieure"]*1000*1000*df_al_vegetale["Pourcentage de protéines"]
disponibilite_interieur_mondiale_kproteine = np.sum(disponibilite_interieur_mondiale_kproteine)
print("La disponibilité intérieur mondiale pour la nourriture végétale est de {} kproteine par an.".format(disponibilite_interieur_mondiale_kproteine))

La disponibilité intérieur mondiale pour la nourriture végétale est de 335873095150.2485 kproteine par an.


### Question 7 
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

Selon la FAO: 
un homme entre 30 et 59,9 ans, ayant une activité physique moyenne (1.75 pal), un poids moyen de 70 kilogrammes a un besoin nutritionnel de 2950 kcalories par jour.
un femme entre 30 et 59,9 ans, ayant une activité physique moyenne (1.75 pal), un poids moyen de 60 kilogrammes a un besoin nutritionnel de 2400 kcalories par jour.
soit une moyenne de 2675 kcal.
Source: http://www.fao.org/docrep/007/y5686e/y5686e08.htm

Pour des adultes en bonne santé, les besoins nutritionnels moyens s'élève à 0,66g de protéine par kilogramme par jour. Pour un adulte de 65 kilogrammes, les besoins nutritionnels s'élèvent donc à 42,9 grammes par jour.
Source: https://ec.europa.eu/jrc/en/health-knowledge-gateway/promotion-prevention/nutrition/protein#_Toctb3

In [21]:
# On regarde seulement la colonne nourriture.
besoin_nutritionnel_kcal = 2675*365 #kcal par an
nombre_humain_kcal = int(disponibilite_interieur_mondiale_kcal/besoin_nutritionnel_kcal)
pourcentage_nombre_humain_kcal = round(nombre_humain_kcal/population_totale*100,2)

besoin_nutritionnel_kproteine = 42.9*365/1000 #kproteine par an
nombre_humain_kproteine = int(disponibilite_interieur_mondiale_kproteine/besoin_nutritionnel_kproteine)
pourcentage_nombre_humain_kproteine = round(nombre_humain_kproteine/population_totale*100,2)
print("Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, {} humains pourraient être nourris selon les recommmandations nutritionnels en calorie. Soit {}% de la population mondiale.".format(nombre_humain_kcal,pourcentage_nombre_humain_kcal))
print("Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, {} humains pourraient être nourris selon les recommmandations nutritionnels en protéine. Soit {}% de la population mondiale.".format(nombre_humain_kproteine,pourcentage_nombre_humain_kproteine))

Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, 13178679082 humains pourraient être nourris selon les recommmandations nutritionnels en calorie. Soit 188.34% de la population mondiale.
Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, 21449889526 humains pourraient être nourris selon les recommmandations nutritionnels en protéine. Soit 306.54% de la population mondiale.


### Question 8
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [22]:
# On combine les colonnes: Nourriture, Aliments pour animaux, et Pertes.
disponibilite_interieur_mondiale_kcal_2 = (df_al_vegetale["Nourriture"] + df_al_vegetale["Aliments pour animaux"] + df_al_vegetale["Pertes"])*1000*1000*df_al_vegetale["Ratio énergie/poids en kcal/kg"]
disponibilite_interieur_mondiale_kcal_2 = np.sum(disponibilite_interieur_mondiale_kcal_2)
disponibilite_interieur_mondiale_kcal_2
print("La disponibilité intérieur mondiale pour l'ensemble de la nourriture végétale est de {} kcal par an.".format(disponibilite_interieur_mondiale_kcal_2))

La disponibilité intérieur mondiale pour l'ensemble de la nourriture végétale est de 8868618451174992.0 kcal par an.


In [23]:
# On combine les colonnes: Nourriture, Aliments pour animaux, et Pertes.
disponibilite_interieur_mondiale_kproteine_2 = (df_al_vegetale["Nourriture"] + df_al_vegetale["Aliments pour animaux"] + df_al_vegetale["Pertes"])*1000*1000*df_al_vegetale["Pourcentage de protéines"]
disponibilite_interieur_mondiale_kproteine_2 = np.sum(disponibilite_interieur_mondiale_kproteine_2)
print("La disponibilité intérieur mondiale pour l'ensemble de la nourriture végétale est de {} kproteine par an.".format(disponibilite_interieur_mondiale_kproteine_2))

La disponibilité intérieur mondiale pour l'ensemble de la nourriture végétale est de 211805451400.80585 kproteine par an.


In [24]:
nombre_humain_kcal = int(disponibilite_interieur_mondiale_kcal_2/besoin_nutritionnel_kcal)
pourcentage_nombre_humain_kcal = round(nombre_humain_kcal/population_totale*100,2)

nombre_humain_kproteine = int(disponibilite_interieur_mondiale_kproteine_2/besoin_nutritionnel_kproteine)
pourcentage_nombre_humain_kproteine = round(nombre_humain_kproteine/population_totale*100,2)

print("Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, {} humains pourraient être nourris selon les recommmandations nutritionnels en calorie. Soit {}% de la population mondiale.".format(nombre_humain_kcal,pourcentage_nombre_humain_kcal))
print("Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, {} humains pourraient être nourris selon les recommmandations nutritionnels en protéine. Soit {}% de la population mondiale.".format(nombre_humain_kproteine,pourcentage_nombre_humain_kproteine))

Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, 9083209270 humains pourraient être nourris selon les recommmandations nutritionnels en calorie. Soit 129.81% de la population mondiale.
Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture, 13526547970 humains pourraient être nourris selon les recommmandations nutritionnels en protéine. Soit 193.31% de la population mondiale.


### Question 9 
Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [25]:
# Nourriture pour les produits végétaux et animaux.
disponibilite_interieur_mondiale_kcal_3 = (df_al["Nourriture"])*1000*1000*df_al["Ratio énergie/poids en kcal/kg"]
disponibilite_interieur_mondiale_kcal_3 = np.sum(disponibilite_interieur_mondiale_kcal_3)
print("La disponibilité intérieur mondiale pour l'ensemble de la nourriture est de {} kcal par an.".format(disponibilite_interieur_mondiale_kcal_3))

La disponibilité intérieur mondiale pour l'ensemble de la nourriture est de 6824130596998004.0 kcal par an.


In [26]:
disponibilite_interieur_mondiale_kproteine_3 = (df_al["Nourriture"])*1000*1000*df_al["Pourcentage de protéines"]
disponibilite_interieur_mondiale_kproteine_3 = np.sum(disponibilite_interieur_mondiale_kproteine_3)
print("La disponibilité intérieur mondiale pour l'ensemble de la nourriture est de {} kproteine par an.".format(disponibilite_interieur_mondiale_kproteine_3))

La disponibilité intérieur mondiale pour l'ensemble de la nourriture est de 207895832799.98373 kproteine par an.


In [27]:
nombre_humain_kcal = int(disponibilite_interieur_mondiale_kcal_3/besoin_nutritionnel_kcal)
pourcentage_nombre_humain_kcal = round(nombre_humain_kcal/population_totale*100,2)

nombre_humain_kproteine = int(disponibilite_interieur_mondiale_kproteine_3/besoin_nutritionnel_kproteine)
pourcentage_nombre_humain_kproteine = round(nombre_humain_kproteine/population_totale*100,2)

print("Avec toute la disponibilité intérieure mondiale en nourriture, {} humains pourraient être nourris selon les recommmandations nutritionnels en calorie. Soit {}% de la population mondiale.".format(nombre_humain_kcal,pourcentage_nombre_humain_kcal))
print("Avec toute la disponibilité intérieure mondiale en nourriture, {} humains pourraient être nourris selon les recommmandations nutritionnels en protéine. Soit {}% de la population mondiale.".format(nombre_humain_kproteine,pourcentage_nombre_humain_kproteine))

Avec toute la disponibilité intérieure mondiale en nourriture, 6989251667 humains pourraient être nourris selon les recommmandations nutritionnels en calorie. Soit 99.88% de la population mondiale.
Avec toute la disponibilité intérieure mondiale en nourriture, 13276867694 humains pourraient être nourris selon les recommmandations nutritionnels en protéine. Soit 189.74% de la population mondiale.


### Question 10 
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [28]:
df_se.head(3)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20112013,2011-2013,millions,7.2,F,Estimation FAO
1,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,8.1,F,Estimation FAO
2,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20132015,2013-2015,millions,9.0,F,Estimation FAO


In [29]:
df_se_2013 = df_se[df_se["Année"]=="2012-2014"]
proportion_sous_alimentee_2013 = (np.sum(df_se_2013["Valeur"]*1000000))/population_totale*100
proportion_sous_alimentee_2013

df_se_2016 = df_se[df_se["Année"]=="2015-2017"]
sous_alimentee_2016 = np.sum(df_se_2016["Valeur"]*1000000)
sous_alimentee_2016

print("En 2013, la proportion de la population sous alimentée était de {}%.".format(round(proportion_sous_alimentee_2013,2)))
print("En 2016, {} humains étaient sous alimentées. N'ayant pas les chiffres de la population pour l'année 2016, nous ne pouvons donner la proportion.".format(sous_alimentee_2016))

En 2013, la proportion de la population sous alimentée était de 12.53%.
En 2016, 868200000.0 humains étaient sous alimentées. N'ayant pas les chiffres de la population pour l'année 2016, nous ne pouvons donner la proportion.


In [30]:
# Liste des produits céréaliers
# On rajoute une colonne 'cereale' à la table origine

liste_cereale=["Blé","Riz (Eq Blanchi)","Orge","Maïs","Seigle","Avoine","Millet","Sorgho","Céréales, Autres"]
# 9 céréales
# 87 produits non-céréales
# 96 produit au total selon la FAO.
i = 0
df_origine["Cereale"]="Attente_valeur" # On créé la colonne Céréale.
for produit in df_origine["Produit"]:
    if produit in liste_cereale:
        df_origine.iloc[i,3]="cereale" # On donne une valeur à la colone cereale.
        i += 1
    else:
        df_origine.iloc[i,3]="non_cereale"
        i += 1
df_origine.reset_index(inplace=True, drop=True)    
df_origine.head()

Élément,Code Produit,Produit,Origine,Cereale
0,2511,Blé,végétale,cereale
1,2513,Orge,végétale,cereale
2,2514,Maïs,végétale,cereale
3,2515,Seigle,végétale,cereale
4,2516,Avoine,végétale,cereale


In [31]:
# On merge la table df_al avec la colonne cereale de df_origine.
df_al = pd.merge(df_al, df_origine[["Code Produit", "Cereale"]], on = ["Code Produit"]  , how = "inner")

### Question 11 : 
En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [32]:
# Colonnes 'Nourriture' et 'Aliments pour animaux'
# filtre 'Cereale'== 'cereale'
# calcul: (Aliments pour animaux/(Nourriture + Aliments pour animaux))*100

In [33]:
df_al_temp = df_al[df_al["Cereale"]=="cereale"]
al_animaux = np.sum(df_al_temp["Aliments pour animaux"])
al_humain = np.sum(df_al_temp["Nourriture"])
proportion_alimentation_animale = (al_animaux / (al_animaux + al_humain))*100
print("{}% des céréales alimentaires (en termes de poids) sont destinées à l'alimentation animale.".format(round(proportion_alimentation_animale, 2)))

45.91% des céréales alimentaires (en termes de poids) sont destinées à l'alimentation animale.


## Question 12
Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition, pour une année choisie.

Repérez les 15 produits les plus exportés par ce groupe de pays sur l'année choisie.

Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné sur l'année choisie)

Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

    le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
    le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)


In [34]:
# Pour l'année 2013 (car nous ne disposons pas des données du bilan alimentaire pour les années suivantes.)
df_se_2013["Sous Alimentees"]= df_se_2013["Valeur"]*1000000
# On remplace les valeurs NaN par 0.
df_se_2013["Sous Alimentees"].fillna(value=0, inplace=True)
df_se_2013.head()

/Users/eddoha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/eddoha/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Sous Alimentees
1,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,8.1,F,Estimation FAO,8100000.0
6,FS,Données de la sécurité alimentaire,202,Afrique du Sud,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,2.6,F,Estimation FAO,2600000.0
11,FS,Données de la sécurité alimentaire,3,Albanie,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,0.2,F,Estimation FAO,200000.0
16,FS,Données de la sécurité alimentaire,4,Algérie,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,1.8,F,Estimation FAO,1800000.0
21,FS,Données de la sécurité alimentaire,79,Allemagne,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,NaN,<2,Moins de 2 millions de personnes sont sous-ali...,0.0


In [35]:
# On merge la colonne Sous alimentees avec df_al.
df_al = pd.merge(df_al, df_se_2013[["Zone","Sous Alimentees"]], how="inner", left_on = ["Pays"], right_on = ["Zone"])
df_al.head()

,Code Pays,Pays,Code Produit,Produit,Année,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Disponibilité totale alimentaire (Kcal/jour),Disponibilité totale de protéines en quantité (Kg/jour),Ratio énergie/poids en kcal/kg,Pourcentage de protéines,Origine,Moyenne Ratio énergie/poids en kcal/kg,Moyenne Pourcentage de protéines,Cereale,Zone,Sous Alimentees
0,1,Arménie,2511,Blé,2013,93.0,0.0,1024.0,130.60,3.60,...,3.048448e+09,90858.04,2860.368946,0.085252,végétale,2848.688281,0.081866,cereale,Arménie,100000.0
1,1,Arménie,2513,Orge,2013,137.0,26.0,0.0,0.00,0.00,...,0.000000e+00,0.00,2528.407544,0.069238,végétale,2528.407544,0.069238,cereale,Arménie,100000.0
2,1,Arménie,2514,Maïs,2013,96.0,0.0,0.0,0.03,0.00,...,0.000000e+00,29.77,3010.278371,0.000000,végétale,3010.278371,0.072271,cereale,Arménie,100000.0
3,1,Arménie,2515,Seigle,2013,1.0,0.0,1.0,0.12,0.00,...,2.977000e+06,59.54,0.000000,0.000000,végétale,2722.431009,0.073895,cereale,Arménie,100000.0
4,1,Arménie,2516,Avoine,2013,4.0,0.0,2.0,0.37,0.03,...,5.954000e+06,267.93,2173.210000,0.097794,végétale,2143.296415,0.084471,cereale,Arménie,100000.0


In [36]:
df_al_temp = df_al[df_al["Sous Alimentees"]>0]
df_al_temp = df_al_temp.loc[:,["Code Pays", "Pays","Code Produit","Produit","Année","Exportations - Quantité", "Sous Alimentees"]]
df_al_temp = df_al_temp.groupby(by=["Code Produit", "Produit","Année"])[["Exportations - Quantité"]].sum()
df_al_temp.sort_values(by=["Exportations - Quantité"], ascending = False, inplace=True)
df_al_temp.reset_index(inplace=True)
produits_exportes = df_al_temp.iloc[0:15,0] # On enregistre le code produit des 15 premiers produits.
df_al_temp.head(15)

,Code Produit,Produit,Année,Exportations - Quantité
0,2577,Huile de Palme,2013,46285.0
1,2514,Maïs,2013,37888.0
2,2532,Manioc,2013,35843.0
3,2805,Riz (Eq Blanchi),2013,32787.0
4,2511,Blé,2013,24841.0
5,2605,"Légumes, Autres",2013,19948.0
6,2615,Bananes,2013,17805.0
7,2625,"Fruits, Autres",2013,14590.0
8,2848,Lait - Excl Beurre,2013,14486.0
9,2555,Soja,2013,14389.0


In [37]:
df_al_temp = df_al.copy()
# On sélectionne les colonnes
columns = ["Code Produit","Produit","Code Pays","Pays","Année",
           "Autres Utilisations","Importations - Quantité","Exportations - Quantité",
          "Disponibilité intérieure","Nourriture","Aliments pour animaux"]
df_al_temp = df_al_temp.loc[:,columns]
# On classe les pays en fonction de leurs importations.
df_al_temp.sort_values(by=["Importations - Quantité"], ascending = False, inplace=True) 
# On filtre les lignes en fonction de l'iterable contenant les 15 produits les plus exportés.
df_al_temp = df_al_temp.loc[df_al_temp["Code Produit"].isin(produits_exportes)]
# On garde les 200 premières lignes.
df_al_temp = df_al_temp.iloc[:200,:]
# On reset l'index
df_al_temp.reset_index(drop=True, inplace=True)
df_al_temp.head()

,Code Produit,Produit,Code Pays,Pays,Année,Autres Utilisations,Importations - Quantité,Exportations - Quantité,Disponibilité intérieure,Nourriture,Aliments pour animaux
0,2555,Soja,41,"Chine, continentale",2013,0.0,63381.0,250.0,75081.0,5072.0,9530.0
1,2532,Manioc,41,"Chine, continentale",2013,7940.0,29046.0,41.0,33590.0,2645.0,22868.0
2,2514,Maïs,110,Japon,2013,288.0,14403.0,1.0,14661.0,1171.0,10964.0
3,2513,Orge,194,Arabie saoudite,2013,0.0,10547.0,0.0,10317.0,0.0,10000.0
4,2511,Blé,59,Égypte,2013,0.0,10331.0,191.0,19341.0,12048.0,4860.0


In [38]:
# On groupe par produit
df_al_temp = df_al_temp.groupby(by=["Code Produit", "Produit"]).sum()
# On calcule le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure
df_al_temp["ratio_autres_disponibilite"] = round(df_al_temp["Autres Utilisations"]/df_al_temp["Disponibilité intérieure"],2)
# On calcule le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)
df_al_temp["ratio_animal_totale"] = round(df_al_temp["Aliments pour animaux"]/(df_al_temp["Aliments pour animaux"]+df_al_temp["Nourriture"]),2)
df_al_temp.head()


,,Code Pays,Année,Autres Utilisations,Importations - Quantité,Exportations - Quantité,Disponibilité intérieure,Nourriture,Aliments pour animaux,ratio_autres_disponibilite,ratio_animal_totale
Code Produit,Produit,,,,,,,,,,
2511,Blé,6344,92598,15270.0,150622.0,106881.0,462177.0,304685.0,99265.0,0.03,0.25
2513,Orge,850,14091,12.0,22387.0,6271.0,29473.0,338.0,20153.0,0.00,0.98
2514,Maïs,3880,58377,182277.0,103447.0,31673.0,703229.0,56023.0,404042.0,0.26,0.88
2532,Manioc,689,10065,20355.0,34655.0,26157.0,67326.0,15257.0,25149.0,0.30,0.62
2555,Soja,2173,30195,524.0,93455.0,41267.0,159570.0,7510.0,13710.0,0.00,0.65


In [39]:
df_al_temp.sort_values(by="ratio_autres_disponibilite",ascending = False)

,,Code Pays,Année,Autres Utilisations,Importations - Quantité,Exportations - Quantité,Disponibilité intérieure,Nourriture,Aliments pour animaux,ratio_autres_disponibilite,ratio_animal_totale
Code Produit,Produit,,,,,,,,,,
2577,Huile de Palme,1483,24156,20528.0,31767.0,22890.0,28586.0,7783.0,0.0,0.72,0.00
2532,Manioc,689,10065,20355.0,34655.0,26157.0,67326.0,15257.0,25149.0,0.30,0.62
2514,Maïs,3880,58377,182277.0,103447.0,31673.0,703229.0,56023.0,404042.0,0.26,0.88
2511,Blé,6344,92598,15270.0,150622.0,106881.0,462177.0,304685.0,99265.0,0.03,0.25
2848,Lait - Excl Beurre,4089,58377,12659.0,90464.0,74781.0,379263.0,331684.0,30502.0,0.03,0.08
2805,Riz (Eq Blanchi),1352,18117,2804.0,13990.0,1126.0,150234.0,122832.0,12644.0,0.02,0.09
2513,Orge,850,14091,12.0,22387.0,6271.0,29473.0,338.0,20153.0,0.00,0.98
2555,Soja,2173,30195,524.0,93455.0,41267.0,159570.0,7510.0,13710.0,0.00,0.65
2601,Tomates,792,10065,0.0,7143.0,2372.0,21277.0,20252.0,0.0,0.00,0.00


In [40]:
df_al_temp.sort_values(by="ratio_animal_totale",ascending = False)

,,Code Pays,Année,Autres Utilisations,Importations - Quantité,Exportations - Quantité,Disponibilité intérieure,Nourriture,Aliments pour animaux,ratio_autres_disponibilite,ratio_animal_totale
Code Produit,Produit,,,,,,,,,,
2513,Orge,850,14091,12.0,22387.0,6271.0,29473.0,338.0,20153.0,0.00,0.98
2514,Maïs,3880,58377,182277.0,103447.0,31673.0,703229.0,56023.0,404042.0,0.26,0.88
2763,Poissons Pelagiques,767,12078,0.0,13445.0,4602.0,16094.0,3937.0,12156.0,0.00,0.76
2555,Soja,2173,30195,524.0,93455.0,41267.0,159570.0,7510.0,13710.0,0.00,0.65
2532,Manioc,689,10065,20355.0,34655.0,26157.0,67326.0,15257.0,25149.0,0.30,0.62
2511,Blé,6344,92598,15270.0,150622.0,106881.0,462177.0,304685.0,99265.0,0.03,0.25
2805,Riz (Eq Blanchi),1352,18117,2804.0,13990.0,1126.0,150234.0,122832.0,12644.0,0.02,0.09
2848,Lait - Excl Beurre,4089,58377,12659.0,90464.0,74781.0,379263.0,331684.0,30502.0,0.03,0.08
2605,"Légumes, Autres",1563,22143,0.0,28177.0,13895.0,87101.0,75711.0,3980.0,0.00,0.05


Pour le premier ratio, les trois produits ayant les plus hautes valeurs sont: huile de palme, le manioc et le maïs.
Pour le second ratio, les trois produits ayant les plus hautes valeurs sont: l'orge, le maïs et les poissons pelagiques.

## Question 13 : 
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [41]:
df_al_temp = df_al[df_al["Pays"]=="États-Unis d'Amérique"]
# On sélectionne les colonnes
columns = ["Code Pays","Pays","Année","Code Produit","Produit",
           "Disponibilité intérieure","Production", "Nourriture","Aliments pour animaux","Origine", "Cereale"]
df_al_temp = df_al_temp.loc[:,columns]
df_al_temp.head()

,Code Pays,Pays,Année,Code Produit,Produit,Disponibilité intérieure,Production,Nourriture,Aliments pour animaux,Origine,Cereale
12987,231,États-Unis d'Amérique,2013,2511,Blé,34051.0,57967.0,25742.0,6196.0,végétale,cereale
12988,231,États-Unis d'Amérique,2013,2513,Orge,4867.0,4683.0,160.0,1440.0,végétale,cereale
12989,231,États-Unis d'Amérique,2013,2514,Maïs,292776.0,353699.0,3917.0,128024.0,végétale,cereale
12990,231,États-Unis d'Amérique,2013,2515,Seigle,347.0,195.0,87.0,184.0,végétale,cereale
12991,231,États-Unis d'Amérique,2013,2516,Avoine,2797.0,1016.0,1265.0,1430.0,végétale,cereale


In [42]:
# Pour ce faire, nous cherchons le ratio: 
#(céréales destinées aux animaux)/(Total des aliments pour animaux(viande et végétal compris))
cereales_destinees_animaux = np.sum(df_al_temp[df_al_temp["Cereale"]=="cereale"]["Aliments pour animaux"])
total_aliments_destines_animaux = np.sum(df_al_temp["Aliments pour animaux"])
ratio = cereales_destinees_animaux/total_aliments_destines_animaux
print("Aux États-Unis d'Amérique,{}% de l'alimentation des animaux sont des céréales.".format(round(ratio, 2)))

# On calcule maintenant le total des aliments destinés aux animaux.
aliments_animaux = np.sum(df_al_temp["Aliments pour animaux"])
print("Aux États-Unis d'Amérique, les aliments pour les animaux représentent {} milliers de tonnes.".format(aliments_animaux))

# Economies que pourrait générer la baisse de production de produits animaux.
# Une baisse de 10% entraine une baisse de 10% des aliments pour les animaux dont 96% sont des céréales.
economie = ratio * aliments_animaux * 0.1
print("Par conséquent, une baisse de 10% de la production de produits animaux entrainerait une baisse de 10% de\
 la nourriture destinés aux animaux dont 96% sont des céréales soit {} milliers de tonnes de céréales.".format(economie))

Aux États-Unis d'Amérique,0.96% de l'alimentation des animaux sont des céréales.
Aux États-Unis d'Amérique, les aliments pour les animaux représentent 146632.0 milliers de tonnes.
Par conséquent, une baisse de 10% de la production de produits animaux entrainerait une baisse de 10% de la nourriture destinés aux animaux dont 96% sont des céréales soit 14009.6 milliers de tonnes de céréales.


## Question 14 : 
En Thaïlande, quel proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [43]:
df_al_temp = df_al[(df_al["Pays"]== "Thaïlande") & (df_al["Produit"]== "Manioc")]
df_al_temp.reset_index(inplace=True, drop=False)
proportion_manioc_exportee = round((df_al_temp.loc[0,"Exportations - Quantité"] / df_al_temp.loc[0,"Production"])*100, 2)
print("La Thaïlande exporte {}% du manioc qu'elle produit.".format(proportion_manioc_exportee))

proportion_sous_alimentee_thaïlande = round((df_al_temp.loc[0,"Sous Alimentees"]/df_al_temp.loc[0, "Population totale"]*100), 2)
print("Pour autant, en Thaïlande, {}% de la population est en sous-nutrition.".format(proportion_sous_alimentee_thaïlande))

La Thaïlande exporte 83.41% du manioc qu'elle produit.
Pour autant, en Thaïlande, 9.1% de la population est en sous-nutrition.


# Reformatage des tables pour SQL


### table "population"
Une table appelée population, contenant la population de chaque pays pour chaque année comprise entre 2012 et la dernière année disponible à la date où vous aurez consulté le site de la FAO. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.

In [44]:
# On importe nos données téléchargées.
df_population_2 = pd.read_csv('population_2012_2013.csv', sep=',')
# On créé une nouvelle colonne: population.
df_population_2["Population"] = df_population_2["Valeur"] * 1000
# On sélectionne nos données.
df_population_2 = df_population_2.loc[:,["Code Pays","Pays","Année","Population"]]
# On change le nom de la colonne Code Pays
df_population_2.rename(columns={"Code Pays": "Code_Pays"}, inplace=True)

df_population_2

,Code_Pays,Pays,Année,Population
0,2,Afghanistan,2012,29825000
1,2,Afghanistan,2013,30552000
2,202,Afrique du Sud,2012,52386000
3,202,Afrique du Sud,2013,52776000
4,3,Albanie,2012,3162000
5,3,Albanie,2013,3173000
6,4,Algérie,2012,38482000
7,4,Algérie,2013,39208000
8,79,Allemagne,2012,82800000
9,79,Allemagne,2013,82727000


## Question 15
Proposez une clé primaire pertinente pour cette table.

Notre clé candidate est la clé possédant les attributs: "Pays" et "Année".
Elle permet d'identifier avec certitude, un tuple, c'est à dire une donnée de notre tableau.
Elle est logique, simple (deux attributs), de type basique et intelligible. De plus, aucun de ces attributs ne risque d'avoir une valeur NULL.
De ce fait, on peut la sélectionner comme clé primaire.

### table dispo_alim
Une table appelée dispo_alim contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les informations suivantes:
        la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
        disponibilité alimentaire en tonnes
        disponibilité alimentaire en Kcal/personne/jour
        disponibilité alimentaire de protéines en g/personne/jour
        disponibilité alimentaire de matières grasses en g/personne/jour
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .


In [52]:
# On importe nos données téléchargées.
df_dispo_alim = pd.read_csv('dispo_alim_2012_2013.csv', sep=',')
# On transpose nos données de la colonne Element vers des colonnes.
# Pour notre pivot table
# Index sera les colonnes de gauche permettant de définir nos lignes
# Values pour définir les colonnes auxquelles on tient
# sum or np.sum pour éviter un average
# column optionnal mais permet de segmenter par cette colonne
# Population dans Population totale
# On ne met pas la colonne unité car sur 4 lignes une seule utile
df_dispo_alim = df_dispo_alim.pivot_table(index=["Code Pays","Pays","Code Produit","Produit","Année"], values='Valeur', aggfunc=sum, columns =["Élément"])
# On restructure notre table en sélectionnant les colonnes.
df_dispo_alim = df_dispo_alim.reset_index()
df_dispo_alim = df_dispo_alim.loc[:,["Pays","Code Pays","Année","Produit","Code Produit","Nourriture",
                                     "Disponibilité alimentaire (Kcal/personne/jour)","Disponibilité de protéines en quantité (g/personne/jour)",
                                     "Disponibilité de matières grasse en quantité (g/personne/jour)"]]

df_dispo_alim = pd.merge(df_dispo_alim,df_origine,on="Produit")
del df_dispo_alim["Cereale"]
df_dispo_alim.head(60)

/Users/eddoha/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Élément,Pays,Code Pays,Année,Produit,Code Produit_x,Nourriture,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité de matières grasse en quantité (g/personne/jour),Code Produit_y,Origine
0,Arménie,1,2012,Blé,2511,377.0,995.0,29.69,NaN,2511,végétale
1,Arménie,1,2013,Blé,2511,389.0,1024.0,30.52,NaN,2511,végétale
2,Afghanistan,2,2012,Blé,2511,4810.0,1378.0,37.16,NaN,2511,végétale
3,Afghanistan,2,2013,Blé,2511,4895.0,1369.0,36.91,NaN,2511,végétale
4,Albanie,3,2012,Blé,2511,440.0,1061.0,33.81,NaN,2511,végétale
5,Albanie,3,2013,Blé,2511,440.0,1056.0,33.64,NaN,2511,végétale
6,Algérie,4,2012,Blé,2511,7179.0,1433.0,43.24,NaN,2511,végétale
7,Algérie,4,2013,Blé,2511,7270.0,1424.0,42.97,NaN,2511,végétale
8,Angola,7,2012,Blé,2511,837.0,282.0,8.55,NaN,2511,végétale
9,Angola,7,2013,Blé,2511,874.0,286.0,8.67,NaN,2511,végétale


## Question 16
Proposez une clé primaire pertinente pour cette table.

Notre clé candidate est la clé possédant les attributs: "Pays" et "Produit". Elle permet d'identifier avec certitude, un tuple, c'est à dire une donnée de notre tableau. Elle est logique, simple (deux attributs), de type basique et intelligible. De plus, aucun de ces attributs ne risque d'avoir une valeur NULL. De ce fait, on peut la sélectionner comme clé primaire

### table equilibre_prod

Une table appelée equilibre_prod contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les quantités suivantes :
disponibilité intérieure
aliments pour animaux
semences
pertes
transformés
nourriture
autres utilisations
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.

In [61]:
df_equi_prod = pd.read_csv('equilibre_prod_2012_2013v2.csv', sep=',')
df_equi_prod = df_equi_prod.pivot_table(index=["Code Pays","Pays","Code Produit","Produit","Année"], values='Valeur', aggfunc=sum, columns =["Élément"])
df_equi_prod = df_equi_prod.reset_index()
df_equi_prod = df_equi_prod.loc[:,["Pays","Code Pays","Année","Produit","Code Produit","Nourriture",
                                     "Disponibilité intérieure","Semences",
                                    "Pertes","Aliments pour animaux","Autres Utilisations","Traitement","Exportations - Quantité"]]
df_equi_prod.fillna(0, inplace = True)
df_equi_prod.head(100)

Élément,Pays,Code Pays,Année,Produit,Code Produit,Nourriture,Disponibilité intérieure,Semences,Pertes,Aliments pour animaux,Autres Utilisations,Traitement,Exportations - Quantité
0,Arménie,1,2013,Blé,2511,389.0,554.0,30.0,32.0,93.0,0.0,10.0,1.0
1,Arménie,1,2013,Orge,2513,0.0,198.0,14.0,15.0,137.0,26.0,7.0,0.0
2,Arménie,1,2013,Maïs,2514,0.0,102.0,0.0,7.0,96.0,0.0,0.0,0.0
3,Arménie,1,2013,Seigle,2515,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Arménie,1,2013,Avoine,2516,1.0,6.0,0.0,0.0,4.0,0.0,0.0,0.0
5,Arménie,1,2013,Millet,2517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Arménie,1,2013,Sorgho,2518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Arménie,1,2013,"Céréales, Autres",2520,6.0,21.0,1.0,1.0,1.0,12.0,0.0,0.0
8,Arménie,1,2013,Pommes de Terre,2531,134.0,672.0,123.0,96.0,225.0,16.0,78.0,9.0
9,Arménie,1,2013,Manioc,2532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Question 17
Proposez une clé primaire pertinente pour cette table.

Notre clé candidate est la clé possédant les attributs: "Pays" et "Produit". Elle permet d'identifier avec certitude, un tuple, c'est à dire une donnée de notre tableau. Elle est logique, simple (deux attributs), de type basique et intelligible. De plus, aucun de ces attributs ne risque d'avoir une valeur NULL. De ce fait, on peut la sélectionner comme clé primaire

### table sous_nutrition
Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays et pour chaque année comprise entre 2012 et la dernière année disponible. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.

In [47]:
df_sous_nutrition = pd.read_csv('sous_alimentee_2012_2017.csv', sep=',')
df_sous_nutrition = df_sous_nutrition.pivot_table(index=["Code zone","Zone","Année"], values='Valeur', aggfunc=sum, columns =["Produit"])
df_sous_nutrition = df_sous_nutrition.reset_index()
df_sous_nutrition = df_sous_nutrition.loc[:,["Zone","Code zone","Année","Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)"]]

df_sous_nutrition.head()

Produit,Zone,Code zone,Année,Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)
0,Arménie,1,2011-2013,0.1
1,Arménie,1,2012-2014,0.1
2,Arménie,1,2013-2015,0.1
3,Arménie,1,2014-2016,0.1
4,Arménie,1,2015-2017,0.1


## Question 18
Proposez une clé primaire pertinente pour cette table.

Notre clé candidate est la clé possédant les attributs: "Zone" et "Code Zone". Elle permet d'identifier avec certitude, un tuple, c'est à dire une donnée de notre tableau. Elle est logique, simple (deux attributs), de type basique et intelligible. De plus, aucun de ces attributs ne risque d'avoir une valeur NULL. De ce fait, on peut la sélectionner comme clé primaire

In [56]:
df_dispo_alim.to_csv("Dispo-Alim.csv", index = False)

In [62]:
df_equi_prod.to_csv("Equi-Prod.csv", index = False)

In [50]:
df_sous_nutrition.to_csv("Sous-Nutri.csv", index = False)

In [51]:
df_population_2.to_csv("Population.csv", index = False)